In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from scipy.linalg import eigh, svd
from scipy import exp
import time
import glob
import PIL
import imageio
import pathlib
import matplotlib.pyplot as plt
import tensorflow as tf
from scipy import linalg
import numpy as np
layers = tf.keras.layers
import os
import cv2
from IPython.display import clear_output
from sklearn.externals import joblib 
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import pyplot
from itertools import cycle, islice
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.preprocessing import MinMaxScaler, StandardScaler
colors = np.array(list(islice(cycle(["#377eb8", "#ff7f00", "#4daf4a",
                                                 "#f781bf", "#a65628", "#984ea3",
                                                 "#999999", "#e41a1c", "#dede00"]),
                                          int(20 + 1))))

C:\Users\ilida\AppData\Roaming\Python\Python37\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
import seaborn as sns
cmap = sns.cubehelix_palette(as_cmap=True)

In [3]:
from IPython.display import clear_output

In [16]:
import pickle

In [4]:
def ToVector(Y):
    a = np.unique(Y)
    yvec = np.zeros((Y.shape[0], a.shape[0]))
    for i in range(0,Y.shape[0]):
        yvec[i,:] = a == Y[i]
    return yvec

In [5]:
def LabelKernel(X,Y):
    X = (np.array(X))
    Y = (np.array(Y))
    L = np.dot(X,Y.T) * 1
    return np.transpose(L);

In [6]:
def decode_img(img):
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return tf.image.resize(img, [256, 256])

def process_path(file_path):
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img

In [7]:
dir_with_labels = "D:\\Labels\\"

df = pd.read_csv(dir_with_labels+"data_picked_for_SKPCA.csv")


In [8]:
index_for_data = df.columns.shape[0]-4096

In [9]:
points_count = df.values.shape[0]

In [11]:
data = df[df.columns[index_for_data:]].values

In [12]:
labels_collection = []

In [13]:
data_scaler = MinMaxScaler((0,1))
data_scaled = data_scaler.fit(data).transform(data)

In [17]:
pickle.dump(data_scaler, open("data_scaler_Model_SKPCA_Liniowy.p","wb"))
pickle.dump(data_scaled, open("data_scaled_Model_SKPCA_Liniowy.p","wb"))

In [18]:
labels_age_scaler = MinMaxScaler((0,1))
labels_age_scaled = df["faceAttributes_age"].values.reshape((data_scaled.shape[0], 1))

ids = df[["imgId","mse"]].values

smile = df["faceAttributes_smile"].values.reshape((data_scaled.shape[0], 1))
labels_collection.append(("smile",[(smile,1)],ids))
labels_collection.append(("age",[(labels_age_scaled,1)],ids))
gender = df[["faceAttributes_gender_female","faceAttributes_gender_male"]].values
labels_collection.append(("gender",[(gender,1)],ids))
beard = df["faceAttributes_facialHair_beard"].values.reshape((data_scaled.shape[0], 1))
labels_collection.append(("facialHair_beard",[(beard,1)],ids))

In [20]:
gamma = 0.25

In [21]:
K = pairwise_kernels(data_scaled, metric="rbf", gamma=gamma)

In [23]:
age = labels_collection[1][1][0][0]
smile= labels_collection[0][1][0][0]
gender = labels_collection[2][1][0][0]
beard= labels_collection[3][1][0][0]

In [24]:
L_age_linear = LabelKernel(age,age)
L_smile_linear =  LabelKernel(smile,smile)
L_gender_linear = LabelKernel(gender,gender)
L_beard_linear =  LabelKernel(beard,beard)

L_linear = L_age_linear+L_smile_linear+L_gender_linear+L_beard_linear

del L_smile_linear
del L_age_linear
del L_gender_linear
del L_beard_linear

In [25]:
from mpl_toolkits.mplot3d import Axes3D

In [26]:
l = data_scaled.shape[0]
H = (np.eye(l, dtype=int))+(-1/l * np.ones(l, dtype=int))
Q = K.dot(H).dot(L_linear).dot(H).dot(K)
eigvals, eigenvec = eigh(Q,K)
del Q
eigenvec = eigenvec[:,::-1]
eigvals = eigvals[::-1]
K_trans = np.transpose(K).dot(eigenvec)
del K


pickle.dump(K_trans, open("K_trans_Model_SKPCA_Liniowy.p","wb"))
pickle.dump(eigenvec, open("eigenvec_Model_SKPCA_Liniowy.p","wb"))

In [27]:
class Model:
    K_after_transformation = []
    age = []
    smile= []
    gender_is_men = []
    beard= []
    ids =[]
    eigvals =[]
    def __init__(self,name):
        self.Name = name
        
        
model = Model("SKPCA liniowy")
model.K_after_transformation = K_trans[:,0:10]
model.ids = ids
model.ids_test = dft[["imgId","mse"]].values
model.eigvals = eigvals[0:10]
model.age = labels_collection[1][1][0][0].flatten()
model.smile= labels_collection[0][1][0][0].flatten()
model.gender_is_men = labels_collection[2][1][0][0][:,1].flatten()
model.beard= labels_collection[3][1][0][0].flatten()

In [28]:
pickle.dump(model, open("Model_SKPCA_Liniowy.p","wb"))